# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from typing import Callable, Union, Any
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [2]:
def gini(x: np.ndarray) -> float:
    """
    Считает коэффициент Джини для массива меток x.
    """
    _, counts = np.unique(x, return_counts=True)
    p = counts / len(x)
    return np.sum(p * (1 - p))


def entropy(x: np.ndarray) -> float:
    """
    Считает энтропию для массива меток x.
    """
    _, counts = np.unique(x, return_counts=True)
    p = counts / len(x)
    return -np.sum(p * np.log2(p))


def gain(left_y: np.ndarray, right_y: np.ndarray,
         criterion: Callable) -> float:
    """
    Считает информативность разбиения массива меток.

    Parameters
    ----------
    left_y : np.ndarray
        Левая часть разбиения.
    right_y : np.ndarray
        Правая часть разбиения.
    criterion : Callable
        Критерий разбиения.
    """
    y = np.concatenate([left_y, right_y])
    return criterion(y) - (len(left_y) * criterion(left_y)
                           + len(right_y) * criterion(right_y)) / len(y)


def get_criterion(name: str) -> Callable:
    name = name.lower()
    if name == 'gini':
        return gini
    assert name == 'entropy'
    return entropy

In [3]:
class Node:
    def __init__(self, criterion: Union[Callable, str]):
        if isinstance(criterion, str):
            criterion = get_criterion(criterion)
        self.criterion = criterion
        self.split_dim = None
        self.label = None
        self.left = None
        self.right = None

    def _assign_class(self, y):
        labels, counts = np.unique(y, return_counts=True)
        i = np.argmax(counts)
        self.label = labels[i]

    @property
    def is_leaf(self):
        return self.left is None and self.right is None

    def split(self, X: np.ndarray, y: np.ndarray, levels_left: int,
              min_samples_leaf: int, max_features: int):
        # check if reached max depth
        if levels_left == 0:
            self._assign_class(y)
            return

        # find best split
        dim_best = None
        gain_best = -np.inf
        for dim in np.random.choice(np.arange(X.shape[1]), size=max_features):
            ix = X[:, dim] == 0  # binary features
            samples_left = np.sum(ix)
            if min(samples_left, len(X) - samples_left) < min_samples_leaf:
                continue  # not enough samples in leaf
            g = gain(y[ix], y[~ix], self.criterion)
            if g > gain_best:
                dim_best = dim
                gain_best = g

        # check if managed to split
        if dim_best is None:
            self._assign_class(y)
            return

        # create leaves
        self.split_dim = dim_best
        ix = X[:, dim_best] == 0
        self.left = Node(self.criterion)
        self.left.split(X[ix, :], y[ix], levels_left - 1,
                        min_samples_leaf, max_features)
        self.right = Node(self.criterion)
        self.right.split(X[~ix, :], y[~ix], levels_left - 1,
                         min_samples_leaf, max_features)

    def _predict(self, x: np.ndarray) -> Any:
        if self.is_leaf:
            return self.label
        if x[self.split_dim] == 0:
            return self.left._predict(x)
        return self.right._predict(x)

    def predict(self, X: np.ndarray) -> np.ndarray:
        predictions = []
        assert X.ndim == 2

        # traverse tree for every sample
        for x in X:
            predictions.append(self._predict(x))
        return np.array(predictions)


class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None,
                 min_samples_leaf=1, max_features="auto"):
        # choose samples with bagging
        inds = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
        self.X = X[inds, :]
        self.y = y[inds]

        # choose features
        if max_features == "auto":
            max_features = int(round(np.sqrt(X.shape[1])))
        assert isinstance(max_features, int) and 0 < max_features <= X.shape[1]

        # create tree and perform splits recursively
        self.root = Node(criterion)
        if max_depth is None:
            max_depth = -1
        self.root.split(self.X, self.y, max_depth, min_samples_leaf, max_features)

    def predict(self, X, soft=False):
        """
        Returns binary class labels (if `soft=False`) or probabilities for
        every sample in 2D array `X`.
        """
        return self.root.predict(X)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1,
                 max_features="auto", n_estimators=10):
        self.estimators = [None] * n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features

    def fit(self, X, y):
        for i in range(len(self.estimators)):
            self.estimators[i] = DecisionTree(
                X=X,
                y=y,
                criterion=self.criterion,
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features
            )

    def predict(self, X):
        predictions = []
        for estimator in self.estimators:
            predictions.append(estimator.predict(X))
        most_frequent = [None] * len(X)
        for i in range(len(X)):
            sample_predictions = np.array([p[i] for p in predictions])
            labels, counts = np.unique(sample_predictions, return_counts=True)
            most_frequent[i] = labels[np.argmax(counts)]
        return np.array(most_frequent)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
def feature_importance(rfc):
    raise NotImplementedError

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [6]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
# print("Importance:", feature_importance(rfc))

Accuracy: 1.0


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [7]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [8]:
X, y_age, y_sex, features = read_dataset("datasets/vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [9]:
rfc = RandomForestClassifier(n_estimators=20, max_features=8)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
#print("Most important features:")
#for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
#    print(str(i+1) + ".", name)

Accuracy: 0.7225725094577553


#### Пол

In [10]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
#print("Most important features:")
#for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
#    print(str(i+1) + ".", name)

Accuracy: 0.8461538461538461


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [11]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(None == y))
print("Importance:", None)

Accuracy: 0.0
Importance: None


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [ ]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [ ]:
print("Accuracy:", np.mean(None == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(None, features, 10)):
    print(str(i+1) + ".", name)

#### Пол

In [ ]:
print("Accuracy:", np.mean(None == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(None, features, 10)):
    print(str(i+1) + ".", name)